<a href="https://colab.research.google.com/github/xingchenzhao/study_deep_learning/blob/master/nin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install d2lzh  # installing d2l
!pip install -U --pre mxnet-cu101mkl  # updating mxnet to at least v1.6

In [0]:
import d2lzh as d2l
from mxnet import gluon, init, nd
from mxnet.gluon import nn

**Network in Network**

Instead of using dense layer, we use 1*1 conv layer for each nin block

In [0]:
def nin_block(num_channels, kernel_size, strides, padding):
  blk = nn.Sequential()
  blk.add(nn.Conv2D(num_channels, kernel_size,
                    strides, padding, activation='relu'),
          nn.Conv2D(num_channels, kernel_size=1, activation='relu'),
          nn.Conv2D(num_channels, kernel_size=1, activation='relu'))
  return blk

In [0]:
net = nn.Sequential()
net.add(nin_block(96,kernel_size=11, strides=4, padding=0),
        nn.MaxPool2D(pool_size=3, strides=2),
        nin_block(256,kernel_size=5, strides=1, padding=2),
        nn.MaxPool2D(pool_size=3, strides=2),
        nin_block(384,kernel_size=3, strides=1, padding=1),
        nn.MaxPool2D(pool_size=3, strides=2), nn.Dropout(0.5),
        nin_block(10,kernel_size=3, strides=1,padding=1),
        nn.GlobalAvgPool2D(),
        nn.Flatten())

In [0]:
X = nd.random.uniform(shape=(1,1,224,224))
net.initialize()
for layer in net:
  X = layer(X)
  print(layer.name, 'output shape:\t', X.shape)

In [14]:
lr, num_epochs, batch_size, ctx = 0.1, 10, 128, d2l.try_gpu()
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate':lr})
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, 
              num_epochs)

training on gpu(0)
epoch 1, loss 2.0916, train acc 0.211, test acc 0.416, time 70.7 sec
epoch 2, loss 1.5695, train acc 0.398, test acc 0.431, time 70.7 sec
epoch 3, loss 1.3895, train acc 0.511, test acc 0.584, time 70.6 sec
epoch 4, loss 1.1871, train acc 0.583, test acc 0.613, time 70.8 sec
epoch 5, loss 1.0356, train acc 0.622, test acc 0.629, time 70.9 sec
epoch 6, loss 1.1066, train acc 0.600, test acc 0.628, time 71.0 sec
epoch 7, loss 0.9857, train acc 0.637, test acc 0.641, time 71.1 sec
epoch 8, loss 0.9572, train acc 0.647, test acc 0.648, time 71.0 sec
epoch 9, loss 0.9410, train acc 0.652, test acc 0.651, time 71.1 sec
epoch 10, loss 0.9255, train acc 0.658, test acc 0.657, time 71.1 sec
